In [3]:
# import tensorflow 
import tensorflow as tf
# import keras
from tensorflow import keras
import numpy as np
model = keras.models.load_model('my_keras_fashion_model.h5')

## load in the fasion MNIST dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train, = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Fine Tuning Hyperparameters 

The good thing about Neural Networks is that they're flexible. The downside is that it's tricky figuring out what combiniation of hyperparameters would be best.

One option is to simply try many combinations of hyperparameters and see which is best on the validation set (or use K-Fold Validation). 

To do so, wrap the keras models in objects that mimic Scikit-Learn regressors. 

(320, O'Reilly)

In [6]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

## Keras Regressor

The keras regressor object is a thin wrapper around the Keras model built using the *build_model()* function defined above. Since hyperparameters weren't defined, it'll use the default hyperparameters defined in the *build_model()* method. Now it can it can be used like a normal sklearn regressor: We can train it using the *fit()* method, then evaulate it using the *score()* method. 


In [5]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\David\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [7]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)

Epoch 1/100
1719/1719 [==============================] - 3s 1ms/step - loss: 0.7860 - accuracy: 0.7423 - val_loss: 0.5728 - val_accuracy: 0.8060
Epoch 2/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5293 - accuracy: 0.8195 - val_loss: 0.4986 - val_accuracy: 0.8364
Epoch 3/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4827 - accuracy: 0.8329 - val_loss: 0.4606 - val_accuracy: 0.8470
Epoch 4/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4573 - accuracy: 0.8420 - val_loss: 0.4486 - val_accuracy: 0.8512
Epoch 5/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4399 - accuracy: 0.8479 - val_loss: 0.4299 - val_accuracy: 0.8596
Epoch 6/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4256 - accuracy: 0.8523 - val_loss: 0.4346 - val_accuracy: 0.8454
Epoch 7/100
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4143 - accuracy: 0.8561 - val_loss: 0.4037 - val_ac

In [10]:
y_pred = keras_reg.predict(X_test[:3])

1/1 [==============================] - 0s 75ms/step


#### Note: any extra paramter passed to the fit() method will get passed to the underlying keras model. The score will also be the opposite of the MSE, as sklearn likes scores, not losses

# HOWEVER...

We don't want to train a single model like this, we want to train hundreds of variants and see which performs best on the validation set. Since there's many hyperparameters, preferable to use a randomized search rather than grid search. 

In [11]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {"n_hidden": [0, 1, 2, 3],
                    "n_neurons": np.arange(1, 100),
                    "learning_rate": reciprocal(3e-4, 3e-2),}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])


Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 1.1609 - accuracy: 0.6030 - val_loss: 0.7294 - val_accuracy: 0.7542
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6606 - accuracy: 0.7782 - val_loss: 0.6016 - val_accuracy: 0.7982
Epoch 3/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.5737 - accuracy: 0.8076 - val_loss: 0.5512 - val_accuracy: 0.8112
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.8187 - val_loss: 0.5091 - val_accuracy: 0.8246
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5027 - accuracy: 0.8272 - val_loss: 0.4956 - val_accuracy: 0.8300
Epoch 6/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.4859 - accuracy: 0.8327 - val_loss: 0.4809 - val_accuracy: 0.8334
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.4715 - accuracy: 0.8373 - val_loss: 0.4666 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 2ms/step - loss: 1.0849 - accuracy: 0.6478 - val_loss: 0.6949 - val_accuracy: 0.7694
Epoch 2/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.6464 - accuracy: 0.7797 - val_loss: 0.5847 - val_accuracy: 0.8046
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5683 - accuracy: 0.8027 - val_loss: 0.5305 - val_accuracy: 0.8240
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5277 - accuracy: 0.8161 - val_loss: 0.5072 - val_accuracy: 0.8270
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5007 - accuracy: 0.8249 - val_loss: 0.4956 - val_accuracy: 0.8352
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.4830 - accuracy: 0.8304 - val_loss: 0.4795 - val_accuracy: 0.8368
Epoch 7/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.4687 - accuracy: 0.8362 - val_loss: 0.4664 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 1.1055 - accuracy: 0.6283 - val_loss: 0.7125 - val_accuracy: 0.7596
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6493 - accuracy: 0.7799 - val_loss: 0.5832 - val_accuracy: 0.7996
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5652 - accuracy: 0.8063 - val_loss: 0.5344 - val_accuracy: 0.8192
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5218 - accuracy: 0.8213 - val_loss: 0.4943 - val_accuracy: 0.8330
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.4954 - accuracy: 0.8314 - val_loss: 0.4804 - val_accuracy: 0.8358
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.4767 - accuracy: 0.8354 - val_loss: 0.4766 - val_accuracy: 0.8400
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.4636 - accuracy: 0.8402 - val_loss: 0.4535 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.9226 - accuracy: 0.7120 - val_loss: 0.6844 - val_accuracy: 0.7902
Epoch 2/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.6455 - accuracy: 0.7929 - val_loss: 0.6002 - val_accuracy: 0.8104
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5855 - accuracy: 0.8117 - val_loss: 0.5582 - val_accuracy: 0.8222
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.8202 - val_loss: 0.5358 - val_accuracy: 0.8268
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 0.8250 - val_loss: 0.5219 - val_accuracy: 0.8280
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5152 - accuracy: 0.8297 - val_loss: 0.5064 - val_accuracy: 0.8364
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5032 - accuracy: 0.8331 - val_loss: 0.4957 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.9356 - accuracy: 0.7050 - val_loss: 0.6976 - val_accuracy: 0.7770
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6504 - accuracy: 0.7877 - val_loss: 0.6043 - val_accuracy: 0.8046
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5877 - accuracy: 0.8086 - val_loss: 0.5657 - val_accuracy: 0.8184
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.8169 - val_loss: 0.5381 - val_accuracy: 0.8272
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.8231 - val_loss: 0.5211 - val_accuracy: 0.8308
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5171 - accuracy: 0.8268 - val_loss: 0.5139 - val_accuracy: 0.8326
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5051 - accuracy: 0.8297 - val_loss: 0.4988 - val_ac

...

Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.9290 - accuracy: 0.7089 - val_loss: 0.6922 - val_accuracy: 0.7830
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6485 - accuracy: 0.7938 - val_loss: 0.6034 - val_accuracy: 0.8098
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5879 - accuracy: 0.8109 - val_loss: 0.5611 - val_accuracy: 0.8200
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.8204 - val_loss: 0.5360 - val_accuracy: 0.8250
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.8255 - val_loss: 0.5199 - val_accuracy: 0.8328
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5167 - accuracy: 0.8291 - val_loss: 0.5045 - val_accuracy: 0.8362
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5044 - accuracy: 0.8330 - val_loss: 0.4948 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.9353 - accuracy: 0.7070 - val_loss: 0.6952 - val_accuracy: 0.7788
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6514 - accuracy: 0.7893 - val_loss: 0.6029 - val_accuracy: 0.8066
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5899 - accuracy: 0.8073 - val_loss: 0.5635 - val_accuracy: 0.8198
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.8183 - val_loss: 0.5358 - val_accuracy: 0.8270
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.8225 - val_loss: 0.5189 - val_accuracy: 0.8312
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5192 - accuracy: 0.8267 - val_loss: 0.5041 - val_accuracy: 0.8364
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5068 - accuracy: 0.8310 - val_loss: 0.5005 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 1ms/step - loss: 0.9328 - accuracy: 0.7077 - val_loss: 0.6891 - val_accuracy: 0.7802
Epoch 2/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.6497 - accuracy: 0.7901 - val_loss: 0.6037 - val_accuracy: 0.8052
Epoch 3/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5877 - accuracy: 0.8083 - val_loss: 0.5615 - val_accuracy: 0.8208
Epoch 4/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.8171 - val_loss: 0.5342 - val_accuracy: 0.8284
Epoch 5/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5328 - accuracy: 0.8232 - val_loss: 0.5187 - val_accuracy: 0.8334
Epoch 6/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5175 - accuracy: 0.8257 - val_loss: 0.5081 - val_accuracy: 0.8344
Epoch 7/100
1146/1146 [==============================] - 1s 1ms/step - loss: 0.5049 - accuracy: 0.8307 - val_loss: 0.4949 - val_ac

Epoch 1/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.9273 - accuracy: 0.6972 - val_loss: 0.6285 - val_accuracy: 0.7878
Epoch 2/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5660 - accuracy: 0.8079 - val_loss: 0.5123 - val_accuracy: 0.8260
Epoch 3/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4999 - accuracy: 0.8291 - val_loss: 0.4769 - val_accuracy: 0.8400
Epoch 4/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4685 - accuracy: 0.8359 - val_loss: 0.4715 - val_accuracy: 0.8396
Epoch 5/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4456 - accuracy: 0.8435 - val_loss: 0.4256 - val_accuracy: 0.8566
Epoch 6/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4301 - accuracy: 0.8510 - val_loss: 0.4565 - val_accuracy: 0.8452
Epoch 7/100
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4168 - accuracy: 0.8550 - val_loss: 0.4261 - val_ac

KeyboardInterrupt: 

In [12]:
# you can now access the best training parameters
rnd_search_cv.best_params_
rnd_search_cv.best_score_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [ ]:
# save the best model  
model = rnd_search_cv.best_estimator_.model
model.save("my_keras_fashion_model_rnd_tuned.h5")


# Mind you, there may be more EFFICIENT methods of optimization than going through parameters randomly and hoping for the best. 
Keras Tuner is one such tool. 

(414, Manning)